In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("surajghuwalewala/ham1000-segmentation-and-classification")

print("Path to dataset files:", path)

100%|██████████| 2.59G/2.59G [00:24<00:00, 113MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/surajghuwalewala/ham1000-segmentation-and-classification/versions/2


In [4]:
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split

In [5]:

dataset_path = "/root/.cache/kagglehub/datasets/surajghuwalewala/ham1000-segmentation-and-classification/versions/2"

# List files in the dataset directory
print("Dataset contents:", os.listdir(dataset_path))


Dataset contents: ['images', 'GroundTruth.csv', 'masks']


In [6]:
# Define dataset path (after downloading)
dataset_path = "/root/.cache/kagglehub/datasets/surajghuwalewala/ham1000-segmentation-and-classification/versions/2"

# Load the metadata file (assumed to be in CSV format)
metadata_path = os.path.join(dataset_path, "GroundTruth.csv")  # Adjust filename if needed
df = pd.read_csv(metadata_path)

# Ensure you have the required columns
print(df.head())

# Convert one-hot encoding to single class column
df['dx'] = df[['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']].idxmax(axis=1)

# Now apply label mapping
label_mapping = {"MEL": 0, "NV": 1, "BCC": 2, "AKIEC": 3, "BKL": 4, "DF": 5, "VASC": 6}
df['dx'] = df['dx'].map(label_mapping)
df = df.dropna(subset=['dx'])  # Remove rows with missing labels

# Define the image directory
image_dir = os.path.join(dataset_path, "images")  # Adjust if needed

# Splitting Data Stratified
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['dx'], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, stratify=train_df['dx'], random_state=42)

# Function to move images to appropriate folders
def move_images(df, source_dir, dest_dir):
    os.makedirs(dest_dir, exist_ok=True)
    for img_name in df['image']:
        src = os.path.join(source_dir, img_name + ".jpg")  # Adjust extension if needed
        dest = os.path.join(dest_dir, img_name + ".jpg")
        if os.path.exists(src):
            shutil.copy(src, dest)

# Create directories
move_images(train_df, image_dir, "dataset/train")
move_images(val_df, image_dir, "dataset/val")
move_images(test_df, image_dir, "dataset/test")

print(f"Train size: {len(train_df)}, Val size: {len(val_df)}, Test size: {len(test_df)}")

          image  MEL   NV  BCC  AKIEC  BKL   DF  VASC
0  ISIC_0024306  0.0  1.0  0.0    0.0  0.0  0.0   0.0
1  ISIC_0024307  0.0  1.0  0.0    0.0  0.0  0.0   0.0
2  ISIC_0024308  0.0  1.0  0.0    0.0  0.0  0.0   0.0
3  ISIC_0024309  0.0  1.0  0.0    0.0  0.0  0.0   0.0
4  ISIC_0024310  1.0  0.0  0.0    0.0  0.0  0.0   0.0
Train size: 7210, Val size: 802, Test size: 2003


In [7]:
# Check for duplicate images between splits to identify leakage
def check_duplicates(df1, df2, key_column):
    common = set(df1[key_column]).intersection(set(df2[key_column]))
    return len(common), common

key_column = "image"  #ID
dup_train_test, dup_train_test_samples = check_duplicates(train_df, test_df, key_column)
dup_train_val, dup_train_val_samples = check_duplicates(train_df, val_df, key_column)
dup_val_test, dup_val_test_samples = check_duplicates(val_df, test_df, key_column)

print(f"Duplicates between Train & Test: {dup_train_test}")
print(f"Duplicates between Train & Validation: {dup_train_val}")
print(f"Duplicates between Validation & Test: {dup_val_test}")

# If duplicates exist, remove them from the test set
if dup_train_test > 0:
    test_df = test_df[~test_df[key_column].isin(dup_train_test_samples)]
if dup_train_val > 0:
    val_df = val_df[~val_df[key_column].isin(dup_train_val_samples)]
if dup_val_test > 0:
    test_df = test_df[~test_df[key_column].isin(dup_val_test_samples)]


Duplicates between Train & Test: 0
Duplicates between Train & Validation: 0
Duplicates between Validation & Test: 0


In [8]:
# Check for patient-level leakage
if 'patient_id' in df.columns:
    train_patients = set(train_df['patient_id'])
    test_patients = set(test_df['patient_id'])
    val_patients = set(val_df['patient_id'])

    overlap_train_test = train_patients.intersection(test_patients)
    overlap_train_val = train_patients.intersection(val_patients)
    overlap_val_test = val_patients.intersection(test_patients)

    print(f"Patient overlap between Train & Test: {len(overlap_train_test)}")
    print(f"Patient overlap between Train & Validation: {len(overlap_train_val)}")
    print(f"Patient overlap between Validation & Test: {len(overlap_val_test)}")

    # Remove patients from test/val that are also in train
    test_df = test_df[~test_df['patient_id'].isin(overlap_train_test)]
    val_df = val_df[~val_df['patient_id'].isin(overlap_train_val)]


In [9]:
# from PIL import Image

#SOMETHING HAPPENS HERE THAT CAUSES ALL IMAGES TO BE CLASSIFIED AS CORRUPT, STILL NEED TO FIX IT
# # Function to check if an image is corrupt
# # def is_corrupt(image_path):
# #     try:
# #         with Image.open(image_path) as img:
# #             img.verify()  # Check if image is valid
# #         return False  # Image is valid
# #     except Exception:
# #         return True  # Image is corrupt
# def is_corrupt(image_path):
#     try:
#         with Image.open(image_path) as img:
#             img.load()  # Load image instead of just verify()
#         return False
#     except Exception as e:
#         print(f"Corrupt image detected: {image_path} - {e}")
#         return True


# # Remove corrupt images from dataset
# def remove_corrupt_images(df, image_dir, image_column):
#     valid_images = []
#     for img_name in df[image_column]:
#         img_path = os.path.join(image_dir, img_name)
#         if os.path.exists(img_path) and not is_corrupt(img_path):
#             valid_images.append(img_name)

#     return df[df[image_column].isin(valid_images)]

# train_df = remove_corrupt_images(train_df, image_dir, key_column)
# val_df = remove_corrupt_images(val_df, image_dir, key_column)
# test_df = remove_corrupt_images(test_df, image_dir, key_column)


In [77]:
import random
from PIL import Image

# Image augmentation functions
def random_rotation(image):
    return image.rotate(random.uniform(-30, 30))

def random_flip(image):
    if random.random() > 0.5:
        return image.transpose(Image.FLIP_LEFT_RIGHT)
    return image

def random_crop(image, output_size=(224, 224)):
    width, height = image.size
    left = random.randint(0, width // 4)
    top = random.randint(0, height // 4)
    right = width - random.randint(0, width // 4)
    bottom = height - random.randint(0, height // 4)
    return image.crop((left, top, right, bottom)).resize(output_size)

# Apply augmentations and save
# def preprocess_and_save(df, source_dir, dest_dir, image_column):
#     os.makedirs(dest_dir, exist_ok=True)
#     saved_count = 0
#     failed_count = 0
#     for img_name in df[image_column]:
#         img_name = str(img_name) + ".jpg"
#         src = os.path.join(source_dir, img_name)
#         dest = os.path.join(dest_dir, img_name)

#         if os.path.exists(src):
#           try:
#             with Image.open(src) as img:

#                 img = img.convert("RGB")  # Ensure image is in correct format
#                 img = random_rotation(img)
#                 img = random_flip(img)
#                 img = random_crop(img, output_size=(224, 224))  # Resize to 224x224
#                 img.save(dest)
#                 saved_count += 1
#           except Exception as e:
#                 print(f"❌ Error processing {src}: {e}")
#                 failed_count += 1
#         else:
#             failed_count += 1
#     print(f"✅ Successfully saved: {saved_count}, ❌ Failed: {failed_count}")
# def preprocess_and_save(df, source_dir, dest_dir, image_column, label_column):
#     os.makedirs(dest_dir, exist_ok=True)

#     saved_count, failed_count = 0, 0
#     for _, row in df.iterrows():
#         img_name = str(row[image_column]) + ".jpg"  # Ensure extension
#         label = str(row[label_column])  # Get class label
#         class_dir = os.path.join(dest_dir, label)  # Class subfolder

#         os.makedirs(class_dir, exist_ok=True)  # Create class folder if not exists

#         src = os.path.join(source_dir, img_name)
#         dest = os.path.join(class_dir, img_name)

#         if os.path.exists(src):
#             try:
#                 with Image.open(src) as img:
#                     img = img.convert("RGB")
#                     img = random_rotation(img)
#                     img = random_flip(img)
#                     img = random_crop(img, output_size=(224, 224))
#                     img.save(dest)
#                     saved_count += 1

#                     # Print every 100 images for debugging
#                     if saved_count % 100 == 0:
#                         print(f"✅ Processed {saved_count}/{len(df)} images")

#             except Exception as e:
#                 print(f"❌ Error processing {src}: {e}")
#                 failed_count += 1
#         else:
#             failed_count += 1

#     print(f"✅ Successfully saved: {saved_count}, ❌ Failed: {failed_count}")

import os
from PIL import Image

def preprocess_and_save(df, source_dir, dest_dir, image_column):
    os.makedirs(dest_dir, exist_ok=True)

    missing_count = 0
    processed_count = 0

    for img_name in df[image_column]:
        src = os.path.join(source_dir, img_name)  # No extension added yet
        dest = os.path.join(dest_dir, img_name)

        # Check if the file exists with different extensions
        if not os.path.exists(src):
            if os.path.exists(src + ".jpg"):
                src += ".jpg"
                dest += ".jpg"
            elif os.path.exists(src + ".png"):
                src += ".png"
                dest += ".png"
            else:
                print(f"❌ Missing during preprocessing: {src}")
                missing_count += 1
                continue  # Skip to the next image

        # Process and save image
        try:
            with Image.open(src) as img:
                img = img.convert("RGB")  # Ensure correct format
                img.save(dest)
                processed_count += 1
        except Exception as e:
            print(f"❌ Error processing {src}: {e}")

    print(f"✅ Processed images: {processed_count}")
    print(f"❌ Total missing images during preprocessing: {missing_count}")

# Re-run for the test set
preprocess_and_save(test_df, "dataset/test", "dataset/preprocessed_test", "image")


# Apply preprocessing
preprocess_and_save(train_df, "dataset/train", "dataset/preprocessed_train", "image", )
preprocess_and_save(test_df, "dataset/test", "dataset/preprocessed_test", "image",)
preprocess_and_save(val_df, "dataset/val", "dataset/preprocessed_val", "image",)

print("Preprocessing complete.")


✅ Processed images: 2003
❌ Total missing images during preprocessing: 0
❌ Missing during preprocessing: dataset/train/ISIC_0031146
❌ Missing during preprocessing: dataset/train/ISIC_0030109
❌ Missing during preprocessing: dataset/train/ISIC_0028940
❌ Missing during preprocessing: dataset/train/ISIC_0025857
❌ Missing during preprocessing: dataset/train/ISIC_0034201
❌ Missing during preprocessing: dataset/train/ISIC_0025726
❌ Missing during preprocessing: dataset/train/ISIC_0027549
❌ Missing during preprocessing: dataset/train/ISIC_0025852
❌ Missing during preprocessing: dataset/train/ISIC_0025402
❌ Missing during preprocessing: dataset/train/ISIC_0028841
❌ Missing during preprocessing: dataset/train/ISIC_0024786
❌ Missing during preprocessing: dataset/train/ISIC_0028928
❌ Missing during preprocessing: dataset/train/ISIC_0030212
❌ Missing during preprocessing: dataset/train/ISIC_0025941
❌ Missing during preprocessing: dataset/train/ISIC_0031169
❌ Missing during preprocessing: dataset/tra

In [72]:
import os

test_images = os.listdir("dataset/test")
print(f"Total test images on disk: {len(test_images)}")
print("First 5 images:", test_images[:5])
print(test_df['image'].head())


Total test images on disk: 2003
First 5 images: ['ISIC_0026217.jpg', 'ISIC_0026557.jpg', 'ISIC_0032290.jpg', 'ISIC_0028825.jpg', 'ISIC_0030446.jpg']
6357    ISIC_0030663.jpg
6963    ISIC_0031269.jpg
3579    ISIC_0027885.jpg
4042    ISIC_0028348.jpg
7871    ISIC_0032177.jpg
Name: image, dtype: object


In [73]:
test_set_images = set(os.listdir("dataset/test"))  # Images in actual test folder
df_images = set(test_df['image'])  # Images listed in CSV

missing_images = df_images - test_set_images  # Files in CSV but NOT on disk
extra_images = test_set_images - df_images  # Files on disk but NOT in CSV

print(f"❌ Images in CSV but missing on disk: {len(missing_images)}")
print(f"✅ Extra images found in folder but NOT in CSV: {len(extra_images)}")

# Show some missing files
print("Some missing files:", list(missing_images)[:10])
print("Some extra files:", list(extra_images)[:10])


❌ Images in CSV but missing on disk: 0
✅ Extra images found in folder but NOT in CSV: 0
Some missing files: []
Some extra files: []


In [75]:
preprocessed_test_images = os.listdir("dataset/preprocessed_test")
print(f"Total preprocessed test images: {len(preprocessed_test_images)}")
import os

test_subfolders = [f.name for f in os.scandir("dataset/preprocessed_test") if f.is_dir()]
print("Subfolders inside preprocessed_test:", test_subfolders)


Total preprocessed test images: 2010
Subfolders inside preprocessed_test: ['5', '4', '3', '0', '6', '1', '2']


In [76]:
from torch.utils.data import Dataset
from PIL import Image
import torch

class CustomDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]['image']
        img_path = os.path.join(self.img_dir, img_name)

        image = Image.open(img_path).convert("RGB")
        label = self.dataframe.iloc[idx]['dx']

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)

# Use custom dataset
test_dataset = CustomDataset(test_df, "dataset/preprocessed_test", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [33]:
# import os
# print(os.listdir("dataset/preprocessed_train"))  # Should list class names
# train_dataset = datasets.ImageFolder(root="dataset/preprocessed_train", transform=transform)
# print("Train samples:", len(train_dataset))  # Should print correct count


In [34]:
# missing_count = 0
# for img_name in train_df['image']:
#     img_name = str(img_name) + ".jpg"  # Ensure extension
#     img_path = os.path.join(image_dir, img_name)
#     if not os.path.exists(img_path):
#         print(f"❌ Missing: {img_path}")
#         missing_count += 1

# print(f"Total missing images in train_df: {missing_count}")


In [35]:
# import os

# source_dir = "/root/.cache/kagglehub/datasets/surajghuwalewala/ham1000-segmentation-and-classification/versions/2/images"
# print("Checking source directory:", source_dir)
# print("Total files in source:", len(os.listdir(source_dir)))

# # Check first few file names
# print("First 5 files:", os.listdir(source_dir)[:5])


In [78]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import os
from PIL import Image
import numpy as np

# Define the CNN model (Modifying from Keras to PyTorch)
class CNN(nn.Module):
    def __init__(self, num_classes=7):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 128),  # Adjust based on input size
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)  # 7 classes in HAM10000
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# Check model parameters
model = CNN(num_classes=7)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")  # Ensure it's < 60M

# If needed, move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


Total parameters: 6479879


In [37]:
# import os

# image_dir = "/root/.cache/kagglehub/datasets/surajghuwalewala/ham1000-segmentation-and-classification/versions/2/images"

# missing_count = 0
# for img_name in df['image']:
#     img_path = os.path.join(image_dir, img_name + ".jpg")  # Adjust extension if needed
#     if not os.path.exists(img_path):
#         print(f"❌ Missing: {img_path}")
#         missing_count += 1

# print(f"Total missing images: {missing_count}")


In [38]:
# import os

# image_dir = "/root/.cache/kagglehub/datasets/surajghuwalewala/ham1000-segmentation-and-classification/versions/2/images"

# if os.path.exists(image_dir):
#     print("✅ Image directory exists")
#     print(f"Total files in folder: {len(os.listdir(image_dir))}")  # Count files
# else:
#     print("❌ Image directory does NOT exist! Check the path.")


In [39]:
# print(f"Before split - dataset size: {len(df)}")
# train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['dx'], random_state=42)
# print(f"Train size: {len(train_df)}, Test size: {len(test_df)}")
# print(f"Before removing corrupt images - Train size: {len(train_df)}")


In [40]:
# corrupt_count = 0
# for img_name in train_df['image']:
#     img_path = os.path.join(image_dir, img_name)
#     if not os.path.exists(img_path) or is_corrupt(img_path):
#         corrupt_count += 1

# print(f"Total images in train set: {len(train_df)}")
# print(f"Images marked as corrupt: {corrupt_count}")


In [41]:
# existing_images = [img for img in train_df['image'] if os.path.exists(os.path.join(image_dir, img))]
# print(f"Existing images in train set: {len(existing_images)}")


In [79]:
#Dataset loading
class SkinLesionDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]['image']
        img_path = os.path.join(self.img_dir, img_name)
        label = self.dataframe.iloc[idx]['dx']  # Replace with correct column name

        # Load image
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load datasets
train_dataset = SkinLesionDataset(train_df, "dataset/preprocessed_train", transform=transform)
val_dataset = SkinLesionDataset(val_df, "dataset/preprocessed_val", transform=transform)
test_dataset = SkinLesionDataset(test_df, "dataset/preprocessed_test", transform=transform)


# print(df['dx'].unique())  # Check if labels are properly assigned
# print(df['dx'].isnull().sum())  # Check if there are missing values


# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [43]:
# import os

# train_path = "dataset/preprocessed_train"
# print("Checking preprocessed train dataset folder...")
# if not os.path.exists(train_path):
#     print("❌ Folder does NOT exist! Check if preprocessing worked.")
# else:
#     print("✅ Folder exists. Number of files:", len(os.listdir(train_path)))


In [44]:
# preprocess_and_save(train_df, image_dir, "dataset/preprocessed_train", "image")
# preprocess_and_save(val_df, image_dir, "dataset/preprocessed_val", "image")
# preprocess_and_save(test_df, image_dir, "dataset/preprocessed_test", "image")


In [45]:
# import os

# missing_count = 0
# for img_name in train_df['image']:
#     img_path = os.path.join("dataset/preprocessed_train", img_name +".jpg")  # Make sure there's an extension
#     if not os.path.exists(img_path):
#         print(f"❌ Missing: {img_path}")
#         missing_count += 1

# print(f"Total missing images: {missing_count}")


In [80]:
import os

preprocessed_train_dir = "dataset/preprocessed_train"  # Adjust path if needed
print("Checking preprocessed train directory:", preprocessed_train_dir)
print("Total processed images:", len(os.listdir(preprocessed_train_dir)))


Checking preprocessed train directory: dataset/preprocessed_train
Total processed images: 7217


In [81]:
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor()])

# Load dataset
train_dataset = datasets.ImageFolder(root="dataset/preprocessed_train", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Check number of images loaded
print("Train samples:", len(train_dataset))


Train samples: 7210


In [82]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.4f}")

print("Training Complete.")


Epoch 1/10, Loss: 1.0267, Train Accuracy: 0.6684
Epoch 2/10, Loss: 0.9272, Train Accuracy: 0.6741
Epoch 3/10, Loss: 0.8796, Train Accuracy: 0.6814
Epoch 4/10, Loss: 0.8575, Train Accuracy: 0.6864
Epoch 5/10, Loss: 0.8337, Train Accuracy: 0.6877
Epoch 6/10, Loss: 0.8140, Train Accuracy: 0.6992
Epoch 7/10, Loss: 0.7950, Train Accuracy: 0.6999
Epoch 8/10, Loss: 0.7873, Train Accuracy: 0.7015
Epoch 9/10, Loss: 0.7611, Train Accuracy: 0.7119
Epoch 10/10, Loss: 0.7385, Train Accuracy: 0.7178
Training Complete.


In [83]:
missing_files = [img for img in test_df['image'] if not os.path.exists(os.path.join(test_image_dir, img))]

print(f"Total missing images: {len(missing_files)}")
if missing_files:
    print("Some missing images:", missing_files[:10])  # Print the first 10 missing files


Total missing images: 0


In [84]:
print(f"Total missing images: {sum([not os.path.exists(os.path.join('dataset/test', img)) for img in test_df['image']])}")


Total missing images: 0


In [85]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Compute Accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Compute Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Compute Precision, Recall, F1-score
report = classification_report(y_true, y_pred, target_names=["MEL", "NV", "BCC", "AKIEC", "BKL", "DF", "VASC"])
print("Classification Report:\n", report)


Test Accuracy: 0.6096
Confusion Matrix:
 [[   0  195    0    0    0    0   28]
 [   0 1215    0    0    1    0  125]
 [   0  100    0    0    0    0    3]
 [   0   64    0    0    0    0    1]
 [   0  215    0    0    0    0    5]
 [   0   22    0    0    0    0    1]
 [   0   22    0    0    0    0    6]]
Classification Report:
               precision    recall  f1-score   support

         MEL       0.00      0.00      0.00       223
          NV       0.66      0.91      0.77      1341
         BCC       0.00      0.00      0.00       103
       AKIEC       0.00      0.00      0.00        65
         BKL       0.00      0.00      0.00       220
          DF       0.00      0.00      0.00        23
        VASC       0.04      0.21      0.06        28

    accuracy                           0.61      2003
   macro avg       0.10      0.16      0.12      2003
weighted avg       0.44      0.61      0.51      2003



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
